# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションに関連するキーワードの簡単な調査（EDA）を行うことを目的としています。主な問題点は、大量のキーワードのデータを分析し、整理して理解しやすくすることです。Notebookでは、キーワードをカテゴリごとに分析し、欠損値や重複データの検出、さらには代替名に関する詳細な調査を行っています。

### 使用された手法とライブラリ
1. **ライブラリ**:
   - `numpy`: 数値計算用のライブラリ。
   - `pandas`: データ操作やデータフレームの管理に使用。
   - `json`: JSONデータの処理に使用。

2. **データの読み込み**:
   - 検索したデータをPython用のスクリプトファイルから読み込み、JSON形式にデコードしてデータフレームに変換。

3. **データ解析**:
   - 表示するキーワードの一覧作成や、カテゴリごとのキーワードの数を把握。
   - 欠損データや重複データを確認し、必要に応じて処理。

4. **代替名分析**:
   - 各都市、国、および名所に関連する代替名の数を分析し、分布を視覚化。

5. **地理的分析**:
   - 大陸別に国や都市の表現を視覚化し、国のリストを取得するために外部データソースを利用。

最終的に、Notebookはキーワードデータセットの理解を深め、分析結果を円グラフなどで視覚的に表現し、データがどのように分布しているかを示しています。また、視覚化を通して、都市や国の表現を確認し、都市の代替名の数や各大陸における国の表現に関する洞察を得ることを目指しています。

---


# 用語概説 
以下は、Jupyter Notebookの内容に関連するが、機械学習・深層学習初心者がつまずきそうな専門用語の簡単な解説です。

1. **EDA (Exploratory Data Analysis)**:
   データの特性や構造を理解するための手法で、データの分布、相関関係、欠損値などを視覚的に確認するプロセスを指します。可視化や統計的手法が用いられます。

2. **JSON (JavaScript Object Notation)**:
   データの交換フォーマットで、軽量で人間にも読みやすい形式です。データをキーと値のペアで表現するため、プログラム間で簡単にデータを送受信することができます。

3. **Pandas**:
   Pythonのデータ分析ライブラリで、データの操作や分析を効率的に行うためのデータ構造（特にデータフレーム）を提供します。CSVやExcelなど、さまざまなデータ形式を扱えます。

4. **データフレーム**:
   Pandasで使用される表形式のデータ構造で、行と列から構成されます。異なるデータ型を持つ列を持つことができ、データの選択、フィルタリング、集計が簡単に行えます。

5. **concat**:
   Pandasにおいて、複数のデータフレームを縦または横に結合するための関数です。分析やデータ処理において、データを一つにまとめる際に便利です。

6. **左外部結合 (Left Join)**:
   2つのデータフレームを結合する際、左側のデータフレームのすべての行を保持し、右側のデータフレームに該当する行があれば、それを追加します。右側に該当行がない場合、NaN（欠損値）になります。

7. **strip()**:
   文字列の前後から指定した文字を削除するメソッドです。通常は空白を削除することに使用されますが、データのクリーンアップに役立ちます。

8. **duplicated()**:
   Pandasのメソッドで、データフレーム内の重複した行を特定し、重複しているかどうかをブール値（True/False）で返します。重複データの削除や分析に活用されます。

9. **プロット (Plot)**:
   データを視覚的に表示する手法で、グラフやチャートを作成することを指します。分析結果を理解しやすくするため、重要なスキルです。

10. **分布 (Distribution)**:
    データのばらつきや傾向を示す概念で、特定の値がどの程度出現するかを示します。データ分析において、正規分布や一様分布など、さまざまな分布が考慮されることがあります。

これらの用語は、あまり専門的でないものから比較的共通のものであり、もしかしたら初心者には馴染みが薄いかもしれません。しかし、実務や深層学習の実践にはおいて、重要かつ理解しておくべき用語です。

---


<a href="https://colab.research.google.com/github/mzaoualim/Kaggle_LLM_20_Questions/blob/main/Quick_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Colabで開く"/></a>

# はじめに
これはKaggleの[LLM 20 Questions](https://www.kaggle.com/competitions/llm-20-questions)コンペティションで提供されているキーワードについての簡単な調査です。

# データとライブラリ

In [ ]:
# このPython 3環境には、多くの便利な分析ライブラリがインストールされています
# これはkaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# 例えば、以下は読み込むためのいくつかの便利なパッケージです

import numpy as np # 線形代数ライブラリ
import pandas as pd # データ処理、CSVファイルの入出力（例: pd.read_csv）

# 入力データファイルは読み取り専用の"../input/"ディレクトリで利用可能です
# 例えば、これを実行すると（実行をクリックするかShift + Enterを押すことで）、入力ディレクトリ内のすべてのファイルがリストされます

import os
for dirname, _, filenames in os.walk('/kaggle/input'):  # '/kaggle/input'ディレクトリのすべてのファイルを探索
    for filename in filenames:
        print(os.path.join(dirname, filename))  # 各ファイルのフルパスを表示

# 現在のディレクトリ（/kaggle/working/）には最大20GBまで書き込むことができ、"Save & Run All"を使用してバージョンを作成すると出力として保存されます
# 一時ファイルは、/kaggle/temp/に書き込むこともできますが、その場合、現在のセッションを超えては保存されません

# キーワード分析

## キーワードの読み込み

In [ ]:
# Pythonの組み込み関数を使用してテキストファイルを読み込みます
f = open("/kaggle/input/llm-20-questions/llm_20_questions/keywords.py", "r")  # 指定したパスのテキストファイルを読み込みモードで開く
print(f.read())  # ファイルの内容を読み込んで表示します
f.close()  # ファイルを閉じてリソースを解放します

In [ ]:
# より詳細なjsonライブラリを使って読み込みます
import json  # JSON操作用のライブラリをインポート

exec(open("/kaggle/input/llm-20-questions/llm_20_questions/keywords.py").read())  # 指定したパスのPythonファイルを実行し、KEYWORDS_JSONという変数を定義します
KEYWORDS_JSON = json.loads(KEYWORDS_JSON)  # KEYWORDS_JSON文字列をJSON形式にデコードします
KEYWORDS_JSON  # デコードされたKEYWORDS_JSONを表示します

In [ ]:
# カテゴリごとのキーワードの合計を表示します

print(len(KEYWORDS_JSON))  # KEYWORDS_JSON内のカテゴリの数を表示します
for category in KEYWORDS_JSON:  # 各カテゴリに対してループを実行します
    print(category["category"], len(category["words"]))  # カテゴリ名と、そのカテゴリに関連するキーワードの数を表示します

## JSONキーワードの処理

In [ ]:
# JSONデータをデータフレームに変換します

# 最初のカテゴリ（例: 国）のキーワードをデータフレームに変換します
country = pd.json_normalize(KEYWORDS_JSON[0]['words'])  
# 二番目のカテゴリ（例: 都市）のキーワードをデータフレームに変換します
city = pd.json_normalize(KEYWORDS_JSON[1]['words'])  
# 三番目のカテゴリ（例: 名所）のキーワードをデータフレームに変換します
landmark = pd.json_normalize(KEYWORDS_JSON[2]['words'])

In [ ]:
country  # 国に関するキーワードを格納したデータフレームを表示します

In [ ]:
city  # 都市に関するキーワードを格納したデータフレームを表示します

In [ ]:
landmark  # 名所に関するキーワードを格納したデータフレームを表示します

In [ ]:
# 欠損しているキーワードと重複しているキーワードを確認します
print('欠損データの合計:\n',
      '国: ',country['keyword'].isna().sum(), '\n',  # 国のデータフレーム内の欠損値の合計を表示
      '都市: ',city['keyword'].isna().sum(), '\n',  # 都市のデータフレーム内の欠損値の合計を表示
      '名所: ',landmark['keyword'].isna().sum(), '\n'  # 名所のデータフレーム内の欠損値の合計を表示
      )

print('重複データの合計:\n',
      '国: ',country['keyword'].duplicated().sum(), '\n',  # 国のデータフレーム内の重複値の合計を表示
      '都市: ',city['keyword'].duplicated().sum(), '\n',  # 都市のデータフレーム内の重複値の合計を表示
      '名所: ',landmark['keyword'].duplicated().sum(), '\n'  # 名所のデータフレーム内の重複値の合計を表示
      )

In [ ]:
# 重複データを調査します
## 名所データセット
landmark['keyword'].value_counts()  # 名所データフレーム内の各キーワードの出現回数をカウントし、表示します

In [ ]:
landmark[landmark['keyword'] == 'mount saint helens']  # キーワードが「mount saint helens」の名所に関するデータをフィルタリングして表示します

In [ ]:
# 重複した行を排除します

## 名所データセット
landmark.drop_duplicates(subset='keyword', keep="last", inplace=True)  # 'keyword'列の重複を排除し、最後の出現を保持します
landmark['keyword'].duplicated().sum()  # 名所データフレーム内の重複値の合計を表示します

In [ ]:
# 重複データを排除します
## 都市データセット

city['keyword'].duplicated().sum()  # 都市データフレーム内の重複値の合計を表示します
city.drop_duplicates(subset='keyword', keep="last", inplace=True)  # 'keyword'列の重複を排除し、最後の出現を保持します
city['keyword'].duplicated().sum()  # 都市データフレーム内の重複値の合計を再度表示します

## ALTS分析

In [ ]:
# ALTS分析
## 複数の代替名を持つ国、都市、名所の分析

### 都市

In [ ]:
# 代替名が最も多い都市
city  # 都市データフレームを表示します（代替名が含まれています）

In [ ]:
# 各都市の代替名の長さを持つ辞書を生成します {都市: 代替名の数}
city_alts_dict = dict()  # 空の辞書を初期化
for i in range(len(city)):  # 都市データフレームをループします
  city_alts_dict[city.iloc[i, 0]] = len(str(city.iloc[i, 1]).strip('[]').split(','))  # 都市名をキーとして、代替名の数をカウントして辞書に追加します

In [ ]:
city_alts_dict  # 各都市とその代替名の数を持つ辞書を表示します

In [ ]:
# 辞書をPandasデータフレームに変換します
city_alts_df = pd.DataFrame.from_dict(city_alts_dict, orient='index', columns=['len_alts'])  # 辞書をデータフレームに変換し、代替名の長さを列として追加します
city_alts_df.reset_index(inplace=True)  # インデックスをリセットして、元の辞書のキーを新しい列にします

In [ ]:
city_alts_df.rename(columns={'index':'city_labels'}, inplace=True)  # インデックス列の名前を 'city_labels' に変更します

In [ ]:
city_alts_df  # 都市の代替名の数を持つデータフレームを表示します

In [ ]:
# ラベルの数で都市をグループ化します
grouped_city_alts_df = city_alts_df.groupby('len_alts').count()  # 'len_alts'でグループ化し、各グループのカウントを取得します
grouped_city_alts_df  # グループ化されたデータフレームを表示します

In [ ]:
# 代替名の数による都市の分布をプロットします
grouped_city_alts_df.plot(
    kind='pie',  # 円グラフを描画します
    title='代替名が単一か複数の都市',  # グラフのタイトル
    ylabel='',  # Y軸のラベルを空に設定
    legend=False,  # 凡例を表示しない
    subplots=True,  # 各サブプロットを個別に表示
    autopct='%1.1f%%'  # パーセンテージを小数点以下1桁で表示
)

リストされた都市の88%以上（278都市）がユニークなラベルを持ち、1つの都市（ロサンゼルス）は7つの代替ラベルで表されています。

### 国

In [ ]:
# 代替名が最も多い国
country  # 国データフレームを表示します（代替名が含まれています）

In [ ]:
# 各国の代替名の長さを持つ辞書を生成します {国: 代替名の数}
country_alts_dict = dict()  # 空の辞書を初期化
for i in range(len(country)):  # 国データフレームをループします
  country_alts_dict[country.iloc[i, 0]] = len(str(country.iloc[i, 1]).strip('[]').split(','))  # 国名をキーとして、代替名の数をカウントして辞書に追加します

country_alts_dict  # 生成した辞書を表示します

In [ ]:
max(country_alts_dict.values())  # 国の代替名の数の中で最大の値を表示します

In [ ]:
# 辞書をPandasデータフレームに変換します
country_alts_df = pd.DataFrame.from_dict(country_alts_dict, orient='index', columns=['len_alts'])  # 辞書をデータフレームに変換し、代替名の長さを列として追加します
country_alts_df.reset_index(inplace=True)  # インデックスをリセットして、元の辞書のキーを新しい列にします
country_alts_df.rename(columns={'index':'country_labels'}, inplace=True)  # インデックス列の名前を 'country_labels' に変更します

In [ ]:
# ラベルの数で国をグループ化します
grouped_country_alts_df = country_alts_df.groupby('len_alts').count()  # 'len_alts'でグループ化し、各グループのカウントを取得します
grouped_country_alts_df  # グループ化されたデータフレームを表示します

In [ ]:
# 代替名の数による国の分布をプロットします
grouped_country_alts_df.plot(
    kind='pie',  # 円グラフを描画します
    title='代替名が単一か複数の国',  # グラフのタイトル
    ylabel='',  # Y軸のラベルを空に設定
    legend=False,  # 凡例を表示しない
    subplots=True,  # 各サブプロットを個別に表示
    autopct='%1.1f%%'  # パーセンテージを小数点以下1桁で表示
)

また、国の97%（195国）がユニークなラベルで表されており、3国が2つのラベル、2国が3つのラベルで表されています。

### 名所

In [ ]:
# 代替名が最も多い名所
landmark  # 名所データフレームを表示します（代替名が含まれています）

In [ ]:
# 各名所の代替名の長さを持つ辞書を生成します {名所: 代替名の数}
landmark_alts_dict = dict()  # 空の辞書を初期化
for i in range(len(landmark)):  # 名所データフレームをループします
  landmark_alts_dict[landmark.iloc[i, 0]] = len(str(landmark.iloc[i, 1]).strip('[]').split(','))  # 名所名をキーとして、代替名の数をカウントして辞書に追加します

landmark_alts_dict  # 生成した辞書を表示します

In [ ]:
max(landmark_alts_dict.values())  # 名所の代替名の数の中で最大の値を表示します

In [ ]:
# 辞書をPandasデータフレームに変換します
landmark_alts_df = pd.DataFrame.from_dict(landmark_alts_dict, orient='index', columns=['len_alts'])  # 辞書をデータフレームに変換し、代替名の長さを列として追加します
landmark_alts_df.reset_index(inplace=True)  # インデックスをリセットして、元の辞書のキーを新しい列にします
landmark_alts_df.rename(columns={'index':'landmark_labels'}, inplace=True)  # インデックス列の名前を 'landmark_labels' に変更します

In [ ]:
# ラベルの数で名所をグループ化します
grouped_landmark_alts_df = landmark_alts_df.groupby('len_alts').count()  # 'len_alts'でグループ化し、各グループのカウントを取得します
grouped_landmark_alts_df  # グループ化されたデータフレームを表示します

In [ ]:
# 代替名の数による名所の分布をプロットします
grouped_landmark_alts_df.plot(
    kind='pie',  # 円グラフを描画します
    title='代替名が単一か複数の名所',  # グラフのタイトル
    ylabel='',  # Y軸のラベルを空に設定
    legend=False,  # 凡例を表示しない
    subplots=True,  # 各サブプロットを個別に表示
    autopct='%1.1f%%'  # パーセンテージを小数点以下1桁で表示
)

再び、名所の大部分（83%または40）はユニークなラベルで表されており、16%（8）は2つの追加ラベルを持っています。

## 地理的分析

In [ ]:
# 地理的分析
## 大陸の表現
## 複数の都市を持つ国

### 大陸別の国の表現

In [ ]:
# 国のリストを読み込みます
country_by_continent = pd.read_html('https://worldpopulationreview.com/country-rankings/list-of-countries-by-continent')[4].copy()  # 指定したURLから国のリストを取得し、データフレームとしてコピーします

In [ ]:
country_by_continent = country_by_continent[['Country', 'Continent']]  # 'Country'と'Continent'の列のみを選択し、データフレームを更新します

In [ ]:
# 大陸別の国
country_by_continent.Continent.value_counts().plot(
    kind='pie',  # 円グラフを描画します
    ylabel='',  # Y軸のラベルを空に設定
    xlabel='',  # X軸のラベルを空に設定
    title='世界の国々の大陸別分布',  # グラフのタイトル
    autopct='%1.1f%%'  # パーセンテージを小数点以下1桁で表示
)

In [ ]:
country_by_continent['Country'] = country_by_continent.Country.str.lower()  # 国名を小文字に変換し、データフレームを更新します

In [ ]:
country_by_continent  # 大陸別の国のデータフレームを表示します

In [ ]:
# 大陸別の国の分布
country_by_continent.Continent.value_counts(normalize=True)  # 大陸別の国の分布を正規化して表示します（割合を計算）

In [ ]:
# 重複した国はあるか？
country_by_continent.Country.duplicated().sum()  # データフレーム内の重複した国の数を表示します

In [ ]:
# 欠損値の確認
country_by_continent.Country.isna().sum()  # 'Country'列内の欠損値の合計を表示します

In [ ]:
# キーワードデータセットにおける大陸の表現
country.merge(
    country_by_continent,  # 'country_by_continent'データフレームと結合
    how='left',  # 左外部結合を行う
    left_on='keyword',  # 'keyword'列を基に結合
    right_on='Country'  # 'Country'列で結合
)['Continent'].value_counts()  # 結合後の'Continent'列の値のカウントを表示する
# .plot(kind='pie')  # （プロット結果を確認したい場合はコメントアウトを解除）

In [ ]:
# キーワードデータセットにおける大陸の表現をプロットします

country.merge(
    country_by_continent,  # 'country_by_continent'データフレームと結合
    how='left',  # 左外部結合を行う
    left_on='keyword',  # 'keyword'列を基に結合
    right_on='Country'  # 'Country'列で結合
)['Continent'].value_counts().plot(kind='pie',  # 円グラフを描画
                                     title='国々の大陸別分布',  # グラフのタイトル
                                     xlabel='',  # X軸のラベルを空に設定
                                     ylabel='',  # Y軸のラベルを空に設定
                                     autopct='%1.1f%%')  # パーセンテージを小数点以下1桁で表示

キーワードデータセットには、各大陸の公平な表現があります。

### 大陸別の都市の表現

In [ ]:
# 世界の都市を収集します
data = pd.DataFrame(columns=['Name', 'Country'])  # 'Name'と'Country'の列を持つ空のデータフレームを作成
for i in range(1, 563):  # 1から562までの範囲でループ
  data = pd.concat([data, pd.read_html(f'https://geokeo.com/database/city/{i}/')[0][['Name', 'Country']]])  # 各ページから都市名と国名を取得し、データフレームに追加します

In [ ]:
data  # 収集した世界の都市のデータフレームを表示します

In [ ]:
# データを保存します
data.to_csv('world_cities.csv')  # 収集したデータを'world_cities.csv'という名前でCSVファイルとして保存します

In [ ]:
# 前処理を行います
data['Name'] = data.Name.str.lower()  # 'Name'列の都市名を小文字に変換し、データフレームを更新します
data  # 前処理後のデータフレームを表示します

In [ ]:
data['Country'] = data.Country.str.lower()  # 'Country'列の国名を小文字に変換し、データフレームを更新します
data  # 更新後のデータフレームを表示します

In [ ]:
city  # 都市に関するデータフレームを表示します

In [ ]:
cities = city.copy()  # 都市データフレームをコピーして'cities'という新しいデータフレームを作成します

In [ ]:
cities  # 'cities'データフレームを表示します

In [ ]:
# キーワードから国名を抽出します
for i in cities.index:  # 'cities'データフレームの各インデックスに対してループします
  cities.loc[i, 'city'] = cities.loc[i, 'keyword'].split(' ')[0]  # 'keyword'列から最初の単語を取り出して'city'列に設定します
cities  # 更新されたデータフレームを表示します

In [ ]:
# 都市の国を抽出します
cities.merge(
    data,  # 'data'データフレームと結合
    how='left',  # 左外部結合を行う
    left_on='city',  # 'city'列を基に結合
    right_on='Name'  # 'Name'列で結合
)['Country'].value_counts()  # 結合後の'Country'列の値のカウントを表示します

In [ ]:
# 国別の都市をプロットします
cities.merge(
    data,  # 'data'データフレームと結合
    how='left',  # 左外部結合を行う
    left_on='city',  # 'city'列を基に結合
    right_on='Name'  # 'Name'列で結合
)['Country'].value_counts().plot(kind='pie',  # 円グラフを描画
                                     title='国別の都市',  # グラフのタイトル
                                     xlabel='',  # X軸のラベルを空に設定
                                     ylabel='',  # Y軸のラベルを空に設定
                                     autopct='%1.1f%%')  # パーセンテージを小数点以下1桁で表示

少し混乱していますが、北アメリカの都市（米国・カナダ・メキシコ）がどのように支配的であるかを示しています（リストされた都市の約1/4）。

In [ ]:
country_by_continent  # 大陸別の国のデータフレームを表示します

In [ ]:
# 都市名に基づいて大陸を抽出します
cities_countries = cities.merge(
                      data,  # 'data'データフレームと結合
                      how='left',  # 左外部結合を行う
                      left_on='city',  # 'city'列を基に結合
                      right_on='Name'  # 'Name'列で結合
                      )

cities_countries['countries'] = cities_countries.Country.str.lower()  # 'Country'列を小文字に変換して'countries'列に保存
cities_countries.drop(columns='Country', inplace=True)  # 'Country'列を削除

cities_countries.merge(
                    country_by_continent,  # 'country_by_continent'データフレームと結合
                    how='left',  # 左外部結合を行う
                    left_on='countries',  # 'countries'列を基に結合
                    right_on='Country'  # 'Country'列で結合
                    )  # 結合結果を表示（表示部分がないので暗黙的）は後の手順で確認可能です

In [ ]:
# 都市に基づいた大陸の表現を示します
cities_countries.merge(
                    country_by_continent,  # 'country_by_continent'データフレームと結合
                    how='left',  # 左外部結合を行う
                    left_on='countries',  # 'countries'列を基に結合
                    right_on='Country'  # 'Country'列で結合
                    )['Continent'].value_counts().plot(kind='pie',  # 円グラフを描画
                                                      title='都市に基づいた大陸の表現',  # グラフのタイトル
                                                      xlabel='',  # X軸のラベルを空に設定
                                                      ylabel='',  # Y軸のラベルを空に設定
                                                      autopct='%1.1f%%')  # パーセンテージを小数点以下1桁で表示

都市の三分の二はヨーロッパとアジアからのもので、他の大陸は十分に表現されていません。

# リソース

[LLM 20 Questions - キーワード](https://www.kaggle.com/code/docxian/llm-20-questions-keywords/notebook)

[[LLM 20 Questions] EDAマップキーワード](https://www.kaggle.com/code/waechter/llm-20-questions-eda-map-keywords)

---

# コメント 

> ## OminousDude
> 
> 非常に役立ちました！ 私のモデルに助けになりました！
> 
> 
> 


---

> ## waechter
> 
> 共有ありがとうございます！ 
> 
> 大陸別の国の追加や[https://worldpopulationreview.com/country-rankings/list-of-countries-by-continent](https://worldpopulationreview.com/country-rankings/list-of-countries-by-continent)、都市の情報を追加したことが気に入りました。[https://geokeo.com/database/city](https://geokeo.com/database/city) 
> 
> 
> 


---

> ## Payam Amanat
> 
> データセットに関する素晴らしい視覚化とEDA [@mzaoualim](https://www.kaggle.com/mzaoualim) 賛成票を投じました
> 
> 
> 
> > ## Mohamed MZAOUALITopic Author
> > 
> > ありがとう！
> > 
> > 
> > 


---

